In [3]:
q = zeros(64, 64, 64)
q_counter = zeros(64, 64, 64)
# Policy 
policy = fill([0, 0, 0], (64, 64, 64))
# Check if the black king is in check
# Returns true or false if black king is in check

function is_legal_move(position, move, board_display=false)
    # Display board if board_display is true
    if board_display
        println(display_board(position))
    end
    # Check legal white king moves
    if move[1] == 1
        # Check for king move into check
        if abs(move[2] - position[3][1]) <= 1 && abs(move[3] - position[3][2]) <= 1
            return false
        end
        # Check for legal king movement
        if abs(position[1][1] - move[2]) <= 1 && abs(position[1][2] - move[3]) <= 1
            if abs(position[1][1] - move[2]) != 0 || abs(position[1][2] - move[3]) != 0
                if position[2] != [move[2], move[3]]
                    return true
                end 
            end
        end
    end
    # Check legal white queen moves
    if move[1] == 2
        # Self Check
        if position[2] == [move[2], move[3]]
            return false
        end
        # Horizontal Check
        if position[2][1] == move[2]
            const_col = 1
            if move[3] < position[2][2]
                const_col = -1
            end
            for i in 1:abs(move[3] - position[2][2])
                if position[1][1] == position[2][1] && position[1][2] == const_col * i + position[2][2]
                    return false
                end
            end
            return true
        # Vertical Check
        elseif move[3] == position[2][2]
            const_row = 1
            if move[2] < position[2][1]
                const_row = -1
            end
            for i in 1:abs(move[2] - position[2][1])
                if position[1][2] == position[2][2] && position[1][1] == const_row * i + position[2][1]
                    return false
                end
            end
            return true
        # Diagonal Check
        elseif abs(move[2] - position[2][1]) == abs(move[3] - position[2][2])
            const_col = 1
            const_row = 1
            if move[3] < position[2][2]
                const_col = -1
            end
            if move[2] < position[2][1]
                const_row = -1
            end
            for i in 1:abs(move[2] - position[2][1])
                if position[1][1] == const_row * i + position[2][1] && position[1][2] == const_col * i + position[2][2]
                    return false
                end
            end
            return true
        end
    end
    if move[1] == 3
        if in_check([position[1], position[2], [move[2], move[3]]])
            return false
        end
        if abs(position[3][1] - move[2]) <= 1 && abs(position[3][2] - move[3]) <= 1
            if abs(position[3][1] - move[2]) != 0 || abs(position[3][2] - move[3]) != 0
                return true
            end
        end
    end

    return false
end
function in_check(position, board_display=false)
    if board_display
        println(display_board(position))
    end
    # Position is a set of three lists: 1 - white king, 2 - white queen, 3 - black king
    # Check if black king can be captured by white king
    if abs(position[3][1] - position[1][1]) <= 1 && abs(position[3][2] - position[1][2]) <= 1
        return true
    end
    if is_legal_move(position, [1, position[3][1], position[3][2]]) || 
                is_legal_move(position, [2, position[3][1], position[3][2]])
        return true
    end
    # Check if black king can be captured by white queen
    # Horizontal check
#     if position[3][1] == position[2][1]
#         # Check if white king is standing between white queen and black king
#         const_col = 1
#         if position[3][2] < position[2][2]
#             const_col = -1
#         end
#         for i in 1:abs(position[3][2] - position[2][2])
#             if position[1][1] == position[2][1] && position[1][2] == const_col * i + position[2][2]
#                 return false
#             end
#         end
#         return true
#     # Vertical Check
#     elseif position[3][2] == position[2][2]
#         const_row = 1
#         if position[3][1] < position[2][1]
#             const_row = -1
#         end
#         for i in 1:abs(position[3][1] - position[2][1])
#             if position[1][2] == position[2][2] && position[1][1] == const_row * i + position[2][1]
#                 return false
#             end
#         end
#         return true
#     # Diagonal Check
#     elseif abs(position[3][1] - position[2][1]) == abs(position[3][2] - position[2][2])
#         const_col = 1
#         const_row = 1
#         if position[3][2] < position[2][2]
#             const_col = -1
#         end
#         if position[3][1] < position[2][1]
#             const_row = -1
#         end
#         for i in 1:abs(position[3][1] - position[2][1])
#             if position[1][1] == const_row * i + position[2][1] && position[1][2] == const_col * i + position[2][2]
#                 return false
#             end
#         end
#         return true
#     end
    return false
end

function display_board(position)
    board = Array{Char}(undef, 8 , 8)
    fill!(board, '\u3000')
    board[position[1][1], position[1][2]] = '\u2654'
    board[position[2][1], position[2][2]] = '\u2655'
    board[position[3][1], position[3][2]] = '\u265a'
    output = "  "
    for h in 1:8
        output *= string(h) * string('\u3000')
    end
    output *= "\n"
    for i in 1:8
        for j in 1:8
            if j == 1
                output *= string(i)
            end
            output *= "|"
            output *= board[i, j]
        end
        output *= "\n"
    end
    return output
end

# Simulate possible moves until 50 moves is reached or until checkmate/stalemate occurs
function episode(policy, ϵ=0.05, board_display=false)
    # Return result, which is either 0 (failure) or 1 (win)
    # Also returns actions taken for each state
    # TODO: Checkmate/stalemate
    # Counter for 50 move draw scenario
    counter = 0
    result = nothing
    game_end = false
    states = []
    actions = []
    # State: [White king, white queen, black king]
    new_state = [[rand(1:8), rand(1:8)], [rand(1:8), rand(1:8)], [rand(1:8), rand(1:8)]]
    while new_state[1] == new_state[3] || new_state[2] == new_state[3] || new_state[1] == new_state[2] ||in_check(new_state)
        new_state = [[rand(1:8), rand(1:8)], [rand(1:8), rand(1:8)], [rand(1:8), rand(1:8)]]
    end
    current_state = new_state
    new_action = []
    while !game_end
        if board_display
            println(display_board(current_state))
        end
        append!(states, current_state)
        # TODO: Create list of all legal moves
        white_legal_moves = []
        black_legal_moves = []
        for i in 1:2
            for j in 1:8
                for k in 1:8
                    # Add list of legal moves
                    if is_legal_move(current_state, [i, j, k])
                        append!(white_legal_moves, [[i, j, k]])
                    end
                end
            end
        end
        new_action = policy[(current_state[1][1] - 1) * 8 + current_state[1][2], (current_state[1][1] - 1) * 8 + current_state[2][2], 
                        (current_state[1][1] - 1) * 8 + current_state[3][2]]
        if new_action == [0, 0, 0] || rand() < ϵ
            new_action = rand(white_legal_moves)
        end
        append!(actions, new_action)
        # TODO: Update board
        current_state[new_action[1]] = [new_action[2], new_action[3]]
        append!(actions, new_action)
        if board_display
            println(display_board(current_state))
        end
        for j in 1:8
            for k in 1:8
                if is_legal_move(current_state, [3, j, k])
                    append!(black_legal_moves, [[3, j, k]])
                end
            end
        end
        if isempty(black_legal_moves)
            if in_check(current_state)
                result = 1
            else
                result = 0
            end
            break
        end
        random_black_action = rand(black_legal_moves)
        current_state[random_black_action[1]] = [random_black_action[2], random_black_action[3]]
        if current_state[2] == current_state[3]
            result = 0
            break
        end 
        counter += 1
        if counter == 50
            game_end = true
            result = 0
        end
    end
    if board_display
        println(display_board(current_state))
    end
    # println(new_action)
    # 1 = white king, 2 = white queen
    # if policy[new_state]
    # println(current_state)
    # result = rand(0:1)
    return states, actions, result
end

s, a, r = episode(policy, 0.05, true)
# println(s)
# println(a)
println(r)
# print(display_board([[6, 4], [2, 1], [8, 7]]))
# println(in_check([[4, 4], [8, 1], [1, 8]], true))
# println(is_legal_move([[4, 2], [4, 3], [1, 4]], [1, 4, 3], true))
# function multiple()
#     for in 1:10e6
#         # Call function, set state, actions, result = function
#         # Loop through all states and actions
#             # Update state counter
#             # Update policy
#     end
# end

  1　2　3　4　5　6　7　8　
1|　|　|　|　|　|　|　|　
2|　|　|　|　|　|　|　|　
3|　|　|　|　|　|　|　|　
4|　|　|　|♚|　|　|　|　
5|　|　|　|　|　|　|　|　
6|　|　|　|　|　|　|　|　
7|　|　|　|　|　|♕|　|　
8|　|　|♔|　|　|　|　|　

  1　2　3　4　5　6　7　8　
1|　|　|　|　|　|　|　|　
2|　|　|　|　|　|　|　|　
3|　|　|　|　|　|　|　|　
4|　|　|　|♚|　|　|　|　
5|　|　|　|　|　|　|　|　
6|　|　|　|　|　|　|　|　
7|　|　|　|　|　|　|　|♕
8|　|　|♔|　|　|　|　|　

  1　2　3　4　5　6　7　8　
1|　|　|　|　|　|　|　|　
2|　|　|　|　|　|　|　|　
3|　|　|　|　|♚|　|　|　
4|　|　|　|　|　|　|　|　
5|　|　|　|　|　|　|　|　
6|　|　|　|　|　|　|　|　
7|　|　|　|　|　|　|　|♕
8|　|　|♔|　|　|　|　|　

  1　2　3　4　5　6　7　8　
1|　|　|　|　|　|　|　|　
2|　|　|　|　|　|　|　|　
3|　|　|　|　|♚|　|　|　
4|　|　|　|　|　|　|　|　
5|　|　|　|　|　|　|　|　
6|　|　|　|　|　|　|　|♕
7|　|　|　|　|　|　|　|　
8|　|　|♔|　|　|　|　|　

  1　2　3　4　5　6　7　8　
1|　|　|　|　|　|　|　|　
2|　|　|　|　|　|　|　|　
3|　|　|　|♚|　|　|　|　
4|　|　|　|　|　|　|　|　
5|　|　|　|　|　|　|　|　
6|　|　|　|　|　|　|　|♕
7|　|　|　|　|　|　|　|　
8|　|　|♔|　|　|　|　|　

  1　2　3　4　5　6　7　8　
1|　|　|　|　|　|　|　|　
2|　|　|　|　|　|　|　|　
3|　|　|　|♚|　|　|　|　
4|　|　|　|　|　|　|　|　
5|　|　|　|　|　|　|　|　
6|　|　|　|　|　|♕|　|　
7|　|　|　|　|　|　|　|　
8|　|　|♔|　|　|　|　|　

  1　2　3　4　5　6　7　